<a href="https://colab.research.google.com/github/shre6701/Programming-BigDataAnalytics/blob/main/FOF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 2: Empirical Asset Pricing in Python

# Step 1: Install Libraries & Import Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant
import pandas_datareader.data as web

# Define sample period
start_date = '2015-01-01'
end_date = '2024-12-31'



# Step 2: Data Loading & Cleaning

In [3]:
# 1. Load CRSP Stock Data
try:
    # Use the filename specified in the assignment
    crsp_subset = pd.read_csv("crsp_monthly_subsample.csv")

    # Workshop logic: Convert date to datetime format
    # Note: If your CSV has standard dates (YYYY-MM-DD), pd.to_datetime works directly.
    # If it has YYYYMM integers like CRSP raw, we would use format='%Y%m'.
    # We assume standard CSV format here but apply MonthEnd to be safe like the workshop.
    crsp_subset['date'] = pd.to_datetime(crsp_subset['date']) + pd.offsets.MonthEnd(0)

except FileNotFoundError:
    print("Error: 'crsp_monthly_subsample.csv' not found. Please upload it.")

# 2. Load Factor Data (Simulating the CSV load from workshop using DataReader)
# In the workshop: ff5_data = pd.read_csv("F-F_Research_Data_5_Factors_2x3.csv")
# Here we download it but structure it EXACTLY like the workshop CSV would look.

ds_5factors = web.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench', start_date, end_date)
ff5_data = ds_5factors[0]
ds_mom = web.DataReader('F-F_Momentum_Factor', 'famafrench', start_date, end_date)
mom_data = ds_mom[0]

# Merge and reset index to make 'date' a column (matching workshop structure)
factor_data = pd.merge(ff5_data, mom_data, left_index=True, right_index=True)
factor_data.reset_index(inplace=True)
factor_data.rename(columns={'Date': 'date'}, inplace=True)

# Workshop logic: Ensure Factor dates are MonthEnd
# The 'date' column contains Period objects after reset_index. Convert them to timestamps.
factor_data['date'] = factor_data['date'].dt.to_timestamp(how='end')

# Rename columns to match workshop/assignment convention
factor_data.rename(columns={
    'Mkt-RF': 'mktrf', 'SMB': 'smb', 'HML': 'hml',
    'RMW': 'rmw', 'CMA': 'cma', 'RF': 'rf', 'Mom   ': 'mom'
}, inplace=True)

# Clean column names (lowercase)
factor_data.columns = [col.strip().lower() for col in factor_data.columns]

# Workshop logic: Convert percentages to decimals
cols_to_convert = ['mktrf', 'smb', 'hml', 'rmw', 'cma', 'mom', 'rf']
factor_data[cols_to_convert] = factor_data[cols_to_convert] / 100.0

print("Data loaded and cleaned.")

/tmp/ipython-input-2332945943.py:19: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ds_5factors = web.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench', start_date, end_date)
/tmp/ipython-input-2332945943.py:19: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ds_5factors = web.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench', start_date, end_date)
/tmp/ipython-input-2332945943.py:21: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ds_mom = web.DataReader('F-F_Momentum_Factor', 'famafrench', start_date, end_date)
/tm

Data loaded and cleaned.


# Step 3: Answer for Q1 & Q2 (Summary Statistics)
This cell computes the simple averages required for Tables B1 and B2.

In [4]:
# Q1: Factor Returns
# Workshop style: select columns explicitly
table_b1 = factor_data[['mktrf', 'smb', 'hml', 'rmw', 'cma', 'mom', 'rf']].mean().to_frame(name='Avg Monthly Ret')
print("\nTable B1: Factor Averages")
print(table_b1)

# Q2: Stock Returns
table_b2 = crsp_subset.groupby('permno')['ret'].mean().to_frame(name='Avg Monthly Ret')
print("\nTable B2: Stock Averages")
print(table_b2.head())



Table B1: Factor Averages
       Avg Monthly Ret
mktrf         0.009820
smb          -0.001664
hml          -0.001841
rmw           0.003903
cma          -0.000866
mom           0.002009
rf            0.001395

Table B2: Stock Averages
        Avg Monthly Ret
permno                 
10026          0.006283
10032          0.014102
10044         -0.006478
10065          0.014551
10104          0.014750


# Step 4: Answer Q3 & Q4 (Excess Returns & Sharpe Ratios)

In [5]:
# Merge on year and month (Workshop Logic)
crsp_subset['year_month'] = crsp_subset['date'].dt.to_period('M')
factor_data['year_month'] = factor_data['date'].dt.to_period('M')

# Perform the merge
# drop 'date' from factors to avoid duplicate columns
crsp_factors = pd.merge(crsp_subset, factor_data.drop(columns=['date']), on='year_month', how='left')

# Compute excess returns (Workshop naming: 'ret_ex')
crsp_factors['ret_ex'] = crsp_factors['ret'] - crsp_factors['rf']

# Q3: Average Excess Returns
# Using the aggregation style from the notebook
return_stats = crsp_factors.groupby('permno').agg(
    mean_ret=('ret', 'mean'),
    mean_ret_ex=('ret_ex', 'mean'),
    std_ret=('ret', 'std')
)

# Add company names (last available name per permno)
last_comnam = crsp_factors.groupby('permno')['comnam'].last()
return_stats = return_stats.merge(last_comnam, on='permno')

print("\nTable B3: Excess Returns (with names)")
print(return_stats[['mean_ret_ex', 'comnam']].head())

# Q4: Annualised Sharpe Ratio
return_stats['sharpe_mon'] = return_stats['mean_ret_ex'] / return_stats['std_ret']
return_stats['sharpe_ann'] = return_stats['sharpe_mon'] * np.sqrt(12)

print("\nTable B4: Annualised Sharpe Ratios")
print(return_stats[['sharpe_ann']].head())



Table B3: Excess Returns (with names)
        mean_ret_ex                           comnam
permno                                              
10026      0.004888           J & J SNACK FOODS CORP
10032      0.012707                      PLEXUS CORP
10044     -0.007873  ROCKY MOUNTAIN CHOC FAC INC NEW
10065      0.013124                 ADAMS EXPRESS CO
10104      0.013355                      ORACLE CORP

Table B4: Annualised Sharpe Ratios
        sharpe_ann
permno            
10026     0.264621
10032     0.573213
10044    -0.254606
10065     0.966446
10104     0.643786


# Step 5: Answers for Portfolios (Q5 - Q7)

In [6]:
# Ensure data is sorted
crsp_factors = crsp_factors.sort_values(by=['date', 'permno'])

# Q6: Value-Weighted Weights
# Get lagged market cap (t-1)
crsp_factors['mkt_cap'] = crsp_factors['prc'].abs() * crsp_factors['shrout']
crsp_factors['mkt_cap_l1'] = crsp_factors.groupby('permno')['mkt_cap'].shift(1)

# Calculate total market cap per date using transform (Workshop Logic)
total_mkt_cap = crsp_factors.groupby('date')['mkt_cap_l1'].transform('sum')
crsp_factors['w_vw'] = crsp_factors['mkt_cap_l1'] / total_mkt_cap

# Q5: Equally-Weighted Weights
# Count number of stocks per date using transform
n_stocks = crsp_factors.groupby('date')['permno'].transform('count')
crsp_factors['w_ew'] = 1 / n_stocks

# Calculate Portfolio Returns
# Logic: Sum(Weight * Return) grouped by Date
# We multiply first, then sum
crsp_factors['ret_vw_contr'] = crsp_factors['ret_ex'] * crsp_factors['w_vw']
crsp_factors['ret_ew_contr'] = crsp_factors['ret_ex'] * crsp_factors['w_ew']

# Create portfolio stats dataframe
portfolio_returns = crsp_factors.groupby('date')[['ret_vw_contr', 'ret_ew_contr']].sum()
portfolio_returns.rename(columns={'ret_vw_contr': 'ret_vw', 'ret_ew_contr': 'ret_ew'}, inplace=True)

# Calculate Sharpe Ratios for Portfolios
ew_mean = portfolio_returns['ret_ew'].mean()
ew_std = portfolio_returns['ret_ew'].std()
ew_sharpe = (ew_mean / ew_std) * np.sqrt(12)

vw_mean = portfolio_returns['ret_vw'].mean()
vw_std = portfolio_returns['ret_vw'].std()
vw_sharpe = (vw_mean / vw_std) * np.sqrt(12)

print(f"EW Portfolio Sharpe: {ew_sharpe:.4f}")
print(f"VW Portfolio Sharpe: {vw_sharpe:.4f}")

# Table B5: Weights for Jan 2017
jan_2017_weights = crsp_factors[crsp_factors['date'] == '2017-01-31'][['permno', 'w_vw']].set_index('permno')
print("\nTable B5: Jan 2017 VW Weights")
print(jan_2017_weights)


EW Portfolio Sharpe: 0.5341
VW Portfolio Sharpe: 1.1133

Table B5: Jan 2017 VW Weights
            w_vw
permno          
10026   0.003449
10032   0.002507
10044   0.000082
10065   0.001705
10104   0.218091
10107   0.664140
10318   0.003685
10333   0.000036
12079   0.073555
21020   0.032749


# Step 6: Answer Regressions Q8

## Q8: Comparing Sharpe Ratios

When comparing the risk-adjusted returns, there is a huge difference between the two portfolio construction. The Equally-Weighted (EW) portfolio produced a Sharpe ratio of 0.5341,and is the range for the individual stocks in the sample. However, the Value-Weighted (VW) portfolio has significantly outperformed, with a Sharpe ratio of 1.1133 which is eventually more than double that of the EW strategy.

**Economic Interpretation:**

We usually expect the portfolios to outperform single stocks due to the Unsystematic risk cancelling them out, decreasing the overall volatility.

However, the Value-Weighted portfolio has significantly outperformed says a lot about the sample composition.From January 2017 (Table B5) we can look at the weights as the portfolio is not diversified but heavily concentrated. Microsoft (10107) makes up 66.4% of the portfolio, and Oracle (10104) is 21.8%. Thus, these two tech giants together accounts for nearly 90% of the VW portfolio's value.

Therefore, the Sharpe ratio of 1.1133 is not just due to diversification,but due to the exceptional performance of the largest companies in this specific basket (Microsoft and Oracle) during this period. The VW method bets big on these basket because of their huge market capitalization. In contrast, the EW portfolio has equal exposure to the smaller,more volatile or less-performing firms (like Permno 10044 or 10333), which reduced the aggregated risk-adjusted return.

# Step 7: Answer Regressions (Q9 & Q10)

In [9]:
# Q9 & Q10 Regressions
target_permnos = [10107, 12079, 21020]

print("--- Regression Analysis ---")

for permno in target_permnos:
    # Subset data
    reg_data = crsp_factors[crsp_factors['permno'] == permno].copy()

    # Drop missing values (Workshop Logic)
    reg_data = reg_data.dropna(subset=['ret_ex', 'mktrf', 'smb', 'hml', 'rmw', 'cma', 'mom'])

    print(f"\nStock Permno: {permno}")

    # --- CAPM ---
    y = reg_data['ret_ex']
    X_capm = add_constant(reg_data['mktrf'])

    # Workshop style: fit model
    model_capm = OLS(y, X_capm).fit()
    print("Figure F1: CAPM Results:")
    print(model_capm.summary())

    # --- FF6 ---
    X_ff6 = add_constant(reg_data[['mktrf', 'smb', 'hml', 'rmw', 'cma', 'mom']])

    model_ff6 = OLS(y, X_ff6).fit()
    print("Figure F2: Fama-French 6 Factor Results:")
    print(model_ff6.summary())


--- Regression Analysis ---

Stock Permno: 10107
Figure F1: CAPM Results:
                            OLS Regression Results                            
Dep. Variable:                 ret_ex   R-squared:                       0.450
Model:                            OLS   Adj. R-squared:                  0.446
Method:                 Least Squares   F-statistic:                     96.71
Date:                Tue, 16 Dec 2025   Prob (F-statistic):           4.97e-17
Time:                        02:06:28   Log-Likelihood:                 197.52
No. Observations:                 120   AIC:                            -391.0
Df Residuals:                     118   BIC:                            -385.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------

## Q9. CAPM Beta Estimates and Interpretation

Microsoft (10107):
*   Beta (0.9244): The coefficient is slightly below 1, suggesting that Microsoft behaves defensively in comparison to the broader market. Microsoft moves roughly 0.92% for every 1% move in the market.
* Alpha (0.0113): The intercept is positive which implies that Microsoft generated anunduly monthly return of roughly 1.13% which cannot be explained by the risk only, indicating strong performance irrespective of market movements.


General Motors (12079):


*  Beta (1.4088): GM shows a high sensitivity to the market movements. As a cyclical auo stock, it syncronous with market volatility i.e. rising faster in booms and falling harder in downturns.
* Alpha (-0.0049): The intercept is negative and statistically negligible. Thus, we cant conclude that GM underperformed on a risk-adjusted basis; its returns are majorly dur to its significant market exposure.


American Airlines (21020):


* Beta (1.4217): Similar to GM, American Airlines also has beta above 1 making it an aggressive stock. This shows the high operational costs and economic sensitivity in the entire airline industry.
*   Alpha (-0.0172): The intercept is negative and partly valuable at the 10% level. Thus, a potential underperformance is expected that market risk does not account for, majorly due to sector-specific slowdown.


## Q10. Fama-French 6-Factor Model Estimates
The 6-factor model provides detailed insight into the drivers of return, often giving the Alpha negligible as we account for specific risks.

Microsoft (10107):


* Economic Interpretation:Microsoft has a striking SMB coefficient of -0.8100. As SMB measures the return of small firms minus big firms, and it justifies  that Microsoft trades as a large-cap stock as it performs well even when small caps lag.


* Statistical Significance: The Size effect is statistically useful ($t = -4.919, p < 0.001$). Crucially, even after managing its size and market risk, the Intercept (Alpha) remains positive at 0.0089 and is statistically significant. Thus, Microsoft's outperformed not just because it's a large company but due to its genuine residual performance that the standard risk factors cannot attribute.






General Motors (12079):
*   Economic Interpretation: General Motors has shown a very positive loading on HML (Value) of 0.5577. Thus, GM is a mature industrial entity with low valuation multiples, making it a Value stock.

* Statistical Significance: The HML coefficient is high, validating its classification as a Value play. Importantly Alpha dropped to -0.0006 making it completely useless. Thus we can conclude that GM's stock is fairly priced for the risks it carries.




American Airlines (21020):


* Economic Interpretation: While American Airlines it has a high Market Beta none of the other style factors (Value, Size, Profitability, Investment, or Momentum) have significant enough coefficients. The HML loading is positive (0.54), showing some fundamental stability, but the standard error is too high to be sure.

* Statistical Significance: Apart from the Market factor, every other p-value is above 0.05. This concludes American Airlines price movements are majorly driven by the broader market and airline-specific news, rather than the standard Fama-French risks.






# Appendix

Code Snippet:
https://colab.research.google.com/drive/1hEsmLqAHX8lqf04eXA0SnfD9UGORnSsh?authuser=0#scrollTo=iSAqiWCUlLmm


